In [2]:
! pip install transformers
! pip install datasets
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 3.8 MB/s 
     |████████████████████████████████| 120 kB 54.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 94.5 MB/s 
     |████████████████████████████████| 115 kB 82.6 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:

from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time
# from torchmetrics import MeanSquaredError
import torch
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [4]:

import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/ERAI Dataset_with_translation


In [6]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('ERAI_Dataset_pairwise_test_with_translation.json')
df = df.dropna(axis = 0)

pd.set_option('display.max_rows', None)

df.reset_index(drop = True,inplace = True)

df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))

df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))


In [7]:
df_unsupervised = pd.read_json('ERAI_Dataset_unsupervised_with_translation.json')

In [8]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [9]:
def generate_input_data(post,mpp_all_value,ml_all_value,max_len):

 
  

  
  
  full_text = []
  attention_mask = []
  
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),  
      'mpp_value': torch.tensor(mpp_all_value.values, dtype=torch.float),
      
      'ml_value':torch.tensor(ml_all_value.values,dtype = torch.float)
  }

    

In [10]:
 class BertNerClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask

    )[1]
    
    output = self.drop(output)
    # self.out(output)
    
    return self.out(output)

In [11]:

class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      
      self.mpp_value = data['mpp_value']
      self.ml_value = data['ml_value']
     

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.mpp_value[index],self.ml_value[index]

In [12]:
def generate_dataloader(post,mpp_all_value,ml_all_value,max_len,batch_size):
  data = generate_input_data(post,mpp_all_value,ml_all_value,max_len)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader


In [13]:
EPOCHS = 15


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss().to(device)


In [14]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device,
  mpp_flag 
  ):
  
  
  test_pred = []
  real_label = []
  

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      if mpp_flag == True:
        targets = targets = data[2].to(device) 
      if mpp_flag == False:
        targets = data[3].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      
      for i in outputs:
        test_pred.append(i.item())
      for i in targets:
        real_label.append(i.item())
      
 
  return test_pred,real_label
     

In [20]:

def train_and_eval(model, 
  train_loader, 
  loss_fn, 
  optimizer,  
  device,
  epoch,
  label_num,
  test_dataloader,
  mpp_flag,
  real_top_mpp,
  real_top_ml

  ):
  model.train()
  average_state = 1000
  for epo in range(epoch):
    
    
    train_loss = []
    
    train_pred = []
    real_train_label  = []
    

    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)

      if mpp_flag == True:
        targets = data[2].to(device)
      if mpp_flag == False:
        targets = data[3].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      
      loss = loss_fn(outputs, targets)
 
      train_loss.append(loss.item())
      
  
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    test_pred,real_label= test_prediction(model, 
      test_dataloader, 
      loss_fn, 
      optimizer,
      device,
      mpp_flag
      )

    pred_top_mpp = []
    pred_top_ml = []
    pred_mmp_average = []
    pred_ml_average = []

    final_train_loss = np.mean(train_loss)
    


    if mpp_flag == True:
      real_mmp_average = np.mean(real_top_mpp[0:8])
      pred_mmp_rank = np.argsort(test_pred)[::-1][0:8]
      for i in pred_mmp_rank:
        pred_mmp_average.append(real_label[i])
      pred_mmp_average = np.mean(pred_mmp_average)
      mmp_difference = pred_mmp_average-real_mmp_average

      if mmp_difference < average_state:
        accuracy_state = mmp_difference
        torch.save(model.state_dict(), 'task2_sing_regres_mpp.bin')
      print(
'Epoch [{}] Train Totol Loss: {:.4f}, REAL AVERAGE MPP: {:.4F}, PRED AVERAGE MPP: {:.4F}'
    .format(epo+1,final_train_loss,real_mmp_average,pred_mmp_average))

    else:
      real_ml_average = np.mean(real_top_ml[0:8])
    
      pred_ml_rank = np.argsort(test_pred)[0:8]

      for i in pred_ml_rank:
        pred_ml_average.append(real_label[i])
      pred_ml_average = np.mean(pred_ml_average)
      ml_difference = real_ml_average-pred_ml_average

      if ml_difference < average_state:
        accuracy_state = ml_difference
        torch.save(model.state_dict(), 'task2_sing_regres_ml.bin')
      print(
'Epoch [{}] Train Totol Loss: {:.4f}, REAL AVERAGE ML: {:.4F}, PRED AVERAGE ML: {:.4F}'
    .format(epo+1,final_train_loss,real_ml_average,pred_ml_average))


    

In [21]:
random_seed = [171,354,550,667,985]
model_name = ['roberta','chi','fin','sec']

for i in model_name:
  if i == 'chi':
    PRETRAINED_MODEL_NAME = "bert-base-chinese"
    tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'roberta':
    PRETRAINED_MODEL_NAME = '/content/drive/MyDrive/ROBERT_4_model.bin'
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
  if i == 'fin':
    PRETRAINED_MODEL_NAME = 'yiyanghkust/finbert-tone'
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'sec':
    PRETRAINED_MODEL_NAME = "nlpaueb/sec-bert-shape"
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)



  for j in random_seed:
    print(i)
    print(j)
    df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
    
    df = df.dropna(axis = 0)

    df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
    df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))

    df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
    df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))
    
    df = shuffle(df,random_state=j)

    train = df[0:156]

    test = df[156:200]

    if i=='chi'or i == 'roberta':
      train_post1 = train['post1']
      train_post2 = train['post2']
      test_post1 = test['post1']
      test_post2 = test['post2']
    else:
      
      train_post1 = train['post1_en']
      train_post2 = train['post2_en']
      test_post1 = test['post1_en']
      test_post2 = test['post2_en']

    
    train_mmp1 = train['MPP1']
    train_mpp2 = train['MPP2']
    train_ml1 = train['ML1']
    train_ml2 = train['ML2']

    all_train_post = pd.concat([train_post1,train_post2],axis=0)
    all_train_mpp = pd.concat([train_mmp1,train_mpp2],axis=0)
    all_train_ml = pd.concat([train_ml1,train_ml2],axis=0)

    max_len = 512
    batch_size = 8
    train_dataloader = generate_dataloader(all_train_post,all_train_mpp,all_train_ml,max_len,batch_size)

    test_mmp1 = test['MPP1']
    test_mpp2 = test['MPP2']
    test_ml1 = test['ML1']
    test_ml2 = test['ML2']



    all_test_post = pd.concat([test_post1,test_post2],axis=0)
    all_test_mpp = pd.concat([test_mmp1,test_mpp2],axis=0)
    all_test_ml = pd.concat([test_ml1,test_ml2],axis=0)

    real_mpp_rank = np.argsort(all_test_mpp.to_list())
    real_mpp_rank= real_mpp_rank[::-1]
    real_ml_rank = np.argsort(all_test_ml.to_list())

    real_top_mpp = []
    real_top_ml = []

    for i,j in zip(real_mpp_rank,real_ml_rank):
      
      real_top_mpp.append(all_test_mpp.tolist()[i])
      real_top_ml.append(all_test_ml.tolist()[j])


    test_dataloader = generate_dataloader(all_test_post,all_test_mpp,all_test_ml,max_len,8)



    label_num = 1
    model_mpp = BertNerClassifier(label_num).to(device)
    model_ml = BertNerClassifier(label_num).to(device)

    optimizer_mpp = AdamW(model_mpp.parameters(), lr=2e-5, correct_bias=False)
    optimizer_ml = AdamW(model_ml.parameters(), lr=2e-5, correct_bias=False)
    mpp_flag = True
    train_and_eval(model_mpp, 
        train_dataloader, 
        loss_fn, 
        optimizer_mpp,  
        device,
        EPOCHS,
        label_num,
        test_dataloader,
        mpp_flag,
        real_top_mpp,
        real_top_ml
    
      )
    mpp_flag = False
    train_and_eval(model_ml, 
        train_dataloader, 
        loss_fn, 
        optimizer_ml,  
        device,
        EPOCHS,
        label_num,
        test_dataloader,
        mpp_flag,
        real_top_mpp,
        real_top_ml
    
      )


roberta
171


Some weights of the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin and are newly initialized: ['bert.pooler.

Epoch [1] Train Totol Loss: 0.0402, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0314
Epoch [2] Train Totol Loss: 0.0023, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0234
Epoch [3] Train Totol Loss: 0.0009, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0221
Epoch [4] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0231
Epoch [5] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0231
Epoch [6] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0250
Epoch [7] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0206
Epoch [8] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0206
Epoch [9] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0206
Epoch [10] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0206
Epoch [11] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0807, PRED AVERAGE MPP: 0.0206
Epoch [12] Train Totol Loss: 0.0008, REAL

Some weights of the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin and are newly initialized: ['bert.pooler.

Epoch [1] Train Totol Loss: 0.0600, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0212
Epoch [2] Train Totol Loss: 0.0012, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0211
Epoch [3] Train Totol Loss: 0.0007, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0158
Epoch [4] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0316
Epoch [5] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [6] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [7] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [8] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [9] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [10] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [11] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1020, PRED AVERAGE MPP: 0.0303
Epoch [12] Train Totol Loss: 0.0008, REAL

...

Epoch [1] Train Totol Loss: 0.0439, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0201
Epoch [2] Train Totol Loss: 0.0028, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0228
Epoch [3] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0179
Epoch [4] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0228
Epoch [5] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0190
Epoch [6] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0190
Epoch [7] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0168
Epoch [8] Train Totol Loss: 0.0009, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0202
Epoch [9] Train Totol Loss: 0.0009, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0197
Epoch [10] Train Totol Loss: 0.0009, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0204
Epoch [11] Train Totol Loss: 0.0010, REAL AVERAGE MPP: 0.0728, PRED AVERAGE MPP: 0.0158
Epoch [12] Train Totol Loss: 0.0010, REAL

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predic

Epoch [1] Train Totol Loss: 0.0323, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0091
Epoch [2] Train Totol Loss: 0.0014, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0330
Epoch [3] Train Totol Loss: 0.0009, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0206
Epoch [4] Train Totol Loss: 0.0010, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0144
Epoch [5] Train Totol Loss: 0.0012, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0135
Epoch [6] Train Totol Loss: 0.0016, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0113
Epoch [7] Train Totol Loss: 0.0022, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0116
Epoch [8] Train Totol Loss: 0.0015, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0161
Epoch [9] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0202
Epoch [10] Train Totol Loss: 0.0007, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0211
Epoch [11] Train Totol Loss: 0.0007, REAL AVERAGE MPP: 0.0869, PRED AVERAGE MPP: 0.0174
Epoch [12] Train Totol Loss: 0.0008, REAL

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predic

Epoch [1] Train Totol Loss: 0.0243, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0232
Epoch [2] Train Totol Loss: 0.0022, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0281
Epoch [3] Train Totol Loss: 0.0007, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0231
Epoch [4] Train Totol Loss: 0.0007, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0170
Epoch [5] Train Totol Loss: 0.0007, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0178
Epoch [6] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0200
Epoch [7] Train Totol Loss: 0.0008, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0200
Epoch [8] Train Totol Loss: 0.0009, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0187
Epoch [9] Train Totol Loss: 0.0010, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0187
Epoch [10] Train Totol Loss: 0.0011, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0236
Epoch [11] Train Totol Loss: 0.0012, REAL AVERAGE MPP: 0.1100, PRED AVERAGE MPP: 0.0277
Epoch [12] Train Totol Loss: 0.0011, REAL

In [ ]:
model_mpp.load_state_dict(torch.load('task2_sing_regres_mpp.bin', map_location=torch.device("cuda:0")))
model_ml.load_state_dict(torch.load('task2_sing_regres_ml.bin', map_location=torch.device("cuda:0")))

In [ ]:

def generate_test_data(post,max_len):
 
  
  full_text = []
  attention_mask = []
  
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),

  }


In [ ]:

class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index]

In [ ]:
def generate_test_dataloader(post,max_len,batch_size):
  data = generate_test_data(post,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader

In [ ]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  device, 
  ):
  
  
  test_pred = []
  real_label = []
  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      
      for i in outputs:
        test_pred.append(i.item())

  return test_pred


In [ ]:
post = df_unsupervised['post_rationale']
test_dataloader = generate_test_dataloader(post,512,16)

In [ ]:
mpp_pred_value = test_prediction(model_mpp, 
  test_dataloader, 
  loss_fn, 
  device
  )

ml_pred_value = test_prediction(model_ml, 
  test_dataloader, 
  loss_fn, 
  device
  )
mpp_pred_value = np.argsort(mpp_pred_value)[::-1]
ml_pred_value = np.argsort(ml_pred_value)

In [ ]:
from pandas.core.frame import DataFrame

mpp=DataFrame(mpp_pred_value)
ml = DataFrame(ml_pred_value)


In [ ]:
mpp.to_json('ERAI_unsupervised_MPP_[aiml]_[1].json')
ml.to_json('ERAI_unsupervised_ML_[aiml]_[1].json')